# Notes: 
    
    - Some predictors are skewed, do we want to fix this before modeling? 
    
    - Potential Inights: 
        - Profit over time, ie
        
    - Some product names go by 10's of ID's, should we drop name or ID? We would have more unique values with ID 
      but it would be more redundant, whereas we could have only ~ 4000 unique names to work with 
            - The product ID seems to correspond to the category and subcategory, for example: 
             
                              TEC-AC-10003033	Technology	Accessories
               
    - There are exactly 2 Customer ID's for each Customer, and they look very similar except one is less a digit, 
      like the following: Michael Chen	['MC-17845' 'MC-7845']. 
            - Does it make sense then to get rid of the Customer ID's and just Use the names? Or to assign 
              one customer ID to each customer rather than 2? 
    

In [113]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy

import matplotlib.pylab as plt
%matplotlib inline

import statsmodels.formula.api as sm
from scipy.stats import skew

from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score
from dmba import plotDecisionTree, gainsChart, liftChart
from dmba import classificationSummary, regressionSummary

from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.impute import SimpleImputer
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz, plot_tree
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.preprocessing import OrdinalEncoder

import statsmodels.tools.tools as stattools

In [53]:
# import the original df and then make a copy for preprocessing so we have the original if needed 
og_df  = pd.read_csv(r'/Users/summerpurschke/Desktop/ADS/Global_Superstore2.csv')

df = og_df.copy()

In [8]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,5/2/13,7/2/13,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,28-01-2013,30-01-2013,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,5/11/13,6/11/13,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [9]:
df.dtypes

Row ID              int64
Order ID           object
Order Date         object
Ship Date          object
Ship Mode          object
Customer ID        object
Customer Name      object
Segment            object
City               object
State              object
Country            object
Postal Code       float64
Market             object
Region             object
Product ID         object
Category           object
Sub-Category       object
Product Name       object
Sales             float64
Quantity            int64
Discount          float64
Profit            float64
Shipping Cost     float64
Order Priority     object
dtype: object

In [10]:
df.shape

(51290, 24)

In [11]:
df.info()

# missing values in postal code, we have city and state information so we can 
    # likely drop postal code with little to no consquences 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row ID          51290 non-null  int64  
 1   Order ID        51290 non-null  object 
 2   Order Date      51290 non-null  object 
 3   Ship Date       51290 non-null  object 
 4   Ship Mode       51290 non-null  object 
 5   Customer ID     51290 non-null  object 
 6   Customer Name   51290 non-null  object 
 7   Segment         51290 non-null  object 
 8   City            51290 non-null  object 
 9   State           51290 non-null  object 
 10  Country         51290 non-null  object 
 11  Postal Code     9994 non-null   float64
 12  Market          51290 non-null  object 
 13  Region          51290 non-null  object 
 14  Product ID      51290 non-null  object 
 15  Category        51290 non-null  object 
 16  Sub-Category    51290 non-null  object 
 17  Product Name    51290 non-null 

In [12]:
df.describe()

,Row ID,Postal Code,Sales,Quantity,Discount,Profit,Shipping Cost
count,51290.00000,9994.000000,51290.000000,51290.000000,51290.000000,51290.000000,51290.000000
mean,25645.50000,55190.379428,246.490581,3.476545,0.142908,28.610982,26.375915
std,14806.29199,32063.693350,487.565361,2.278766,0.212280,174.340972,57.296804
min,1.00000,1040.000000,0.444000,1.000000,0.000000,-6599.978000,0.000000
25%,12823.25000,23223.000000,30.758625,2.000000,0.000000,0.000000,2.610000
50%,25645.50000,56430.500000,85.053000,3.000000,0.000000,9.240000,7.790000
75%,38467.75000,90008.000000,251.053200,5.000000,0.200000,36.810000,24.450000
max,51290.00000,99301.000000,22638.480000,14.000000,0.850000,8399.976000,933.570000


In [18]:
# Confirm skewed predictors 

print('Sales Skew:', skew(df['Sales'], axis = 0, bias = True ))
print('Quantity Skew:', skew(df['Quantity'], axis = 0, bias = True ))
print('Discount Skew:', skew(df['Discount'], axis = 0, bias = True ))
print('Profit Skew:', skew(df['Profit'], axis = 0, bias = True ))
print('Shipping Cost Skew:', skew(df['Shipping Cost'], axis = 0, bias = True ))

Sales Skew: 8.137842017336732
Quantity Skew: 1.3603279457897046
Discount Skew: 1.3877339656893208
Profit Skew: 4.157066952869827
Shipping Cost Skew: 5.863054951522988


In [19]:
df.columns = [d.replace(' ', '_') for d in df.columns]
df.columns = [d.replace('-', '_') for d in df.columns]
df.head()

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,City,State,...,Product_ID,Category,Sub_Category,Product_Name,Sales,Quantity,Discount,Profit,Shipping_Cost,Order_Priority
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,5/2/13,7/2/13,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,28-01-2013,30-01-2013,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,5/11/13,6/11/13,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [20]:
df.isnull().sum(axis = 0)

Row_ID                0
Order_ID              0
Order_Date            0
Ship_Date             0
Ship_Mode             0
Customer_ID           0
Customer_Name         0
Segment               0
City                  0
State                 0
Country               0
Postal_Code       41296
Market                0
Region                0
Product_ID            0
Category              0
Sub_Category          0
Product_Name          0
Sales                 0
Quantity              0
Discount              0
Profit                0
Shipping_Cost         0
Order_Priority        0
dtype: int64

In [21]:
df['Order_Date'] = pd.to_datetime(df['Order_Date'], infer_datetime_format=True)
df['Ship_Date'] = pd.to_datetime(df['Ship_Date'], infer_datetime_format=True)

In [22]:
df.head()

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,City,State,...,Product_ID,Category,Sub_Category,Product_Name,Sales,Quantity,Discount,Profit,Shipping_Cost,Order_Priority
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,2013-05-02,2013-07-02,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,2013-05-11,2013-06-11,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Row_ID          51290 non-null  int64         
 1   Order_ID        51290 non-null  object        
 2   Order_Date      51290 non-null  datetime64[ns]
 3   Ship_Date       51290 non-null  datetime64[ns]
 4   Ship_Mode       51290 non-null  object        
 5   Customer_ID     51290 non-null  object        
 6   Customer_Name   51290 non-null  object        
 7   Segment         51290 non-null  object        
 8   City            51290 non-null  object        
 9   State           51290 non-null  object        
 10  Country         51290 non-null  object        
 11  Postal_Code     9994 non-null   float64       
 12  Market          51290 non-null  object        
 13  Region          51290 non-null  object        
 14  Product_ID      51290 non-null  object        
 15  Ca

In [24]:
df = df.drop(columns=['Customer_Name', 'Postal_Code'])

In [25]:
#df.describe()

In [26]:
df['Ship_Mode']= df['Ship_Mode'].astype('category')
df['Segment']= df['Segment'].astype('category')
df['Country']= df['Country'].astype('category')
df['Market']= df['Market'].astype('category')
df['Region']= df['Region'].astype('category')
df['Category']= df['Category'].astype('category')
df['Sub_Category']= df['Sub_Category'].astype('category')
df['Order_Priority']= df['Order_Priority'].astype('category')

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Row_ID          51290 non-null  int64         
 1   Order_ID        51290 non-null  object        
 2   Order_Date      51290 non-null  datetime64[ns]
 3   Ship_Date       51290 non-null  datetime64[ns]
 4   Ship_Mode       51290 non-null  category      
 5   Customer_ID     51290 non-null  object        
 6   Segment         51290 non-null  category      
 7   City            51290 non-null  object        
 8   State           51290 non-null  object        
 9   Country         51290 non-null  category      
 10  Market          51290 non-null  category      
 11  Region          51290 non-null  category      
 12  Product_ID      51290 non-null  object        
 13  Category        51290 non-null  category      
 14  Sub_Category    51290 non-null  category      
 15  Pr

In [50]:
# This is how many unique Order_ID's there are if I'm not mistaken?
df.groupby(['Country']).count()[['Order_ID']]  

,Order_ID
Country,
Afghanistan,55
Albania,16
Algeria,196
Angola,122
Argentina,390
...,...
Venezuela,194
Vietnam,265
Yemen,30


In [49]:
df.groupby(['City']).count()[['Order_ID']]

,Order_ID
City,
Aachen,17
Aalen,1
Aalst,4
Aba,25
Abadan,11
...,...
Zwedru,1
Zwickau,3
Zwolle,2


In [83]:
biggest_procurement = df.groupby(['Country']).sum()[['Quantity']].nlargest(n=10, columns=['Quantity'])
biggest_procurement

,Quantity
Country,
United States,37873
France,10804
Australia,10673
Mexico,10011
Germany,7745
China,7081
United Kingdom,6161
Brazil,6148
India,5758


In [90]:
most_profitable_product = df.groupby(['Product_Name']).sum()[['Profit']].nlargest(n=5, columns=['Profit'])
most_profitable_product

,Profit
Product_Name,
Canon imageCLASS 2200 Advanced Copier,25199.9280
"Cisco Smart Phone, Full Size",17238.5206
"Motorola Smart Phone, Full Size",17027.1130
"Hoover Stove, Red",11807.9690
"Sauder Classic Bookcase, Traditional",10672.0730


In [85]:
most_profitable_country = df.groupby(['Country']).sum()[['Profit']].nlargest(n=5, columns=['Profit'])
most_profitable_country

,Profit
Country,
United States,286397.0217
China,150683.0850
India,129071.8350
United Kingdom,111900.1500
France,109029.0030


### Reasons to drop
    - Ship_date, Ship_Mode: we have no information on how profits or satistifaction is effected by the shipping 
        time or class 
    - Postal_Code: a ton of missing values, not worth imputing since we have city, state, country information 
    - should we drop Product ID or Product Name?
    - should we drop Customer ID or Customer Name?

In [187]:
# Make a copy of the df so I can manipulate and drop data 
df_copy = df.copy()

# Make sure that there is one unqiue row ID for each row in the df before reindexing using Row ID 
print('There are',df_copy['Row ID'].nunique(),'Row ID values, and', df_copy.shape, 'Rows x Columns')

# Re-Index using row ID 
df_copy = df_copy.set_index('Row ID')
df_copy.head()

There are 51290 Row ID values, and (51290, 24) Rows x Columns


,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
Row ID,,,,,,,,,,,,,,,,,,,,,
32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
26341,IN-2013-77878,5/2/13,7/2/13,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
13524,ES-2013-1579342,28-01-2013,30-01-2013,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,Germany,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
47221,SG-2013-4320,5/11/13,6/11/13,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,Senegal,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [188]:
# Drop unneccesary or redundant columns 
df_copy = df_copy.drop(columns = ['Ship Date','Ship Mode', 'Postal Code'])

### How many product ID's are there? How many Product Names?

In [189]:
print('There are', df_copy['Product ID'].nunique(), 'unique product IDs, and', 
      df_copy['Product Name'].nunique(), ' unique product names')


# There are only 3788 Unique products, but some go by different ID's

group = pd.DataFrame(df_copy.groupby('Product Name')['Product ID'].unique())
group['Product ID'] = group['Product ID'].astype('str')
group['# Product IDs'] = group['Product ID'].str.split(' ').apply(len)

products  = group.sort_values(by = ['# Product IDs'], ascending = False)
products.head()

There are 10292 unique product IDs, and 3788  unique product names


,Product ID,# Product IDs
Product Name,,
Staples,['OFF-EN-10004773' 'OFF-EN-10003286' 'OFF-PA-1...,46
"Stockwell Paper Clips, Assorted Sizes",['OFF-FA-10002017' 'OFF-FA-10004265' 'OFF-FA-1...,16
"Acco Index Tab, Clear",['OFF-BI-10002738' 'OFF-BI-10002386' 'OFF-BI-1...,12
"HP Copy Machine, Color",['TEC-CO-10004563' 'TEC-CO-10003901' 'TEC-HP -...,12
"Stockwell Push Pins, 12 Pack",['OFF-FA-10002577' 'OFF-FA-10002790' 'OFF-FA-1...,11


### How many customer ID's are there? How many customer Names?

In [190]:
print('There are', df_copy['Customer ID'].nunique(), 'unique Customer IDs, and', 
      df_copy['Customer Name'].nunique(), ' unique Customer names')


# There are only 795 Unique customers, each having 2 customer ID's that look similar 

group1 = pd.DataFrame(df_copy.groupby('Customer Name')['Customer ID'].unique())
group1['Customer ID'] = group1['Customer ID'].astype('str')
group1['# Customer IDs'] = group1['Customer ID'].str.split(' ').apply(len)

customers  = group1.sort_values(by = ['# Customer IDs'], ascending = False)
customers.tail()

There are 1590 unique Customer IDs, and 795  unique Customer names


,Customer ID,# Customer IDs
Customer Name,,
Erica Smith,['ES-14020' 'ES-4020'],2
Erin Ashbrook,['EA-14035' 'EA-4035'],2
Erin Creighton,['EC-14050' 'EC-4050'],2
Erin Mull,['EM-14065' 'EM-4065'],2
Zuschuss Donatelli,['ZD-21925' 'ZD-11925'],2


### Change Order Priority to Ordinal Values 

In [191]:
df_copy['Order Priority'].unique()

dict = {'Critical': 4, 'Medium' : 3, 'High' : 2, 'Low': 1}
df_copy = df_copy.replace({'Order Priority' : dict})
df_copy

,Order ID,Order Date,Customer ID,Customer Name,Segment,City,State,Country,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
Row ID,,,,,,,,,,,,,,,,,,,,
32298,CA-2012-124891,31-07-2012,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,US,East,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,4
26341,IN-2013-77878,5/2/13,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,APAC,Oceania,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,4
25330,IN-2013-71249,17-10-2013,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,APAC,Oceania,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,3
13524,ES-2013-1579342,28-01-2013,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,Germany,EU,Central,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,3
47221,SG-2013-4320,5/11/13,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,Senegal,Africa,Africa,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29002,IN-2014-62366,19-06-2014,KE-16420,Katrina Edelman,Corporate,Kure,Hiroshima,Japan,APAC,North Asia,OFF-FA-10000746,Office Supplies,Fasteners,"Advantus Thumb Tacks, 12 Pack",65.100,5,0.0,4.5000,0.01,3
35398,US-2014-102288,20-06-2014,ZC-21910,Zuschuss Carroll,Consumer,Houston,Texas,United States,US,Central,OFF-AP-10002906,Office Supplies,Appliances,Hoover Replacement Belt for Commercial Guardsm...,0.444,1,0.8,-1.1100,0.01,3
40470,US-2013-155768,2/12/13,LB-16795,Laurel Beltran,Home Office,Oxnard,California,United States,US,West,OFF-EN-10001219,Office Supplies,Envelopes,"#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes",22.920,3,0.0,11.2308,0.01,2


In [194]:
df_copy.head()

,Order ID,Order Date,Customer ID,Customer Name,Segment,City,State,Country,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
Row ID,,,,,,,,,,,,,,,,,,,,
32298,CA-2012-124891,31-07-2012,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,US,East,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,4
26341,IN-2013-77878,5/2/13,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,APAC,Oceania,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,4
25330,IN-2013-71249,17-10-2013,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,APAC,Oceania,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,3
13524,ES-2013-1579342,28-01-2013,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,Germany,EU,Central,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,3
47221,SG-2013-4320,5/11/13,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,Senegal,Africa,Africa,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,4


In [198]:
# Calculate profits per year/ per month 
df_copy['Customer Name'].nunique()

df_copy['State'].nunique()

1094

In [ ]:
# How much profit per product? per Customer? per Area? Etc? 